In [ ]:
# Función de coste binaria con valores {-1, 1}
# g(z) = "\
#     17.5*z_0 - 24*z_1 - 41*z_2 - 23.5*z_3 + 18*z_4 \
#     + 20*z_0*z_1 - 20*z_0*z_2 - 20*z_0*z_3 + 20*z_1*z_2 \
#         - 20*z_1*z_4 + 20*z_2*z_3 - 20*z_2*z_4 + 20*z_3*z_4 \
#     + 20*z_0**2 + 20*z_1**2 + 20*z_2**2 + 20*z_3**2 + 20*z_4**2 + 33 "

# Grafo
edges = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
num_nodes = 4

# Hp = 17.5*Z0 - 24*Z1 - 41*Z2 - 23.5*Z3 + 18*Z4
#     + 20*(Z0*Z1 - Z0*Z2 - Z0*Z3 + Z1*Z2 - Z1*Z4 + Z2*Z3 - Z2*Z4 + Z3*Z4)

# Coeficientes lineales de Hp (17.5 * Z0 - 24 * Z1...)
linear_coefs = [17.5, -24, -41, -23.5, 18]

# Coeficientes cuadráticos Ej: {(2, 3): 20} -> 20 * Z2 * Z3
quadra_coefs = {(0, 1): 20, (1, 2): 20, (2, 3): 20, (0, 2): -20,
                (0, 3): -20, (3, 4): 20, (1, 4): -20, (2, 4): -20}

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Imprime un grafo dirigido pesado
# Entrada
#     graph_nodes: Lista de nodos del grafo
#     graph_edges: Diccionario {vértice: peso} donde vértice es (n1, n2)
#     layout:      Para especificar la posición de los nodos
def print_graph(graph_nodes, graph_edges, layout=None):
    G = nx.DiGraph()

    # Nodes
    G.add_nodes_from(graph_nodes)

    # Edges
    for pair, weight in edges.items():
        G.add_edge(pair[0], pair[1], weight=weight)

    # Print graph
    if layout is None:
        layout = nx.spring_layout(G)

    plt.figure(3, figsize=(6, 2))
    nx.draw(G, layout, with_labels=True, node_size=600)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)
    plt.show()

print_graph(range(num_nodes), edges, layout={0: [-1, 0], 1: [0, 1],
                                             2: [0, -1], 3: [1, 0]})

In [ ]:
# Función de coste binaria con valores {0, 1}
cost_function = "\
5*x_01 + 8*x_02 + 2*x_12 + 7*x_13 + 4*x_23 + \
40*(x_01 + x_02 - 1)**2 + \
40*(x_13 + x_23 - 1)**2 + \
40*(x_01 - x_12 - x_13)**2 + \
40*(x_02 + x_12 - x_23)**2"

# Evalúa una solución concreta según la función de coste para la versión QUBO
# Entrada:
#     - bits: Cadena con los valores de los qubits medidos
#             Orden: q_4 q_3 q_2 q_1 q_0
def eval_cost_function(bits):
    assert len(edges) == len(bits), "Error in cost_function: Length of bits"

    inv_bits = bits[::-1]  # Los qubits están en orden inverso

    param_dict = {}
    # Asignación entre x_ij -> q_n
    # Establecido por el orden de edges. Ej: x_12 -> q_2 (esto es, qubit nº2)
    for n, (i, j) in enumerate(edges):
        param_dict[f"x_{i}{j}"] = int(inv_bits[n])

    return eval(cost_function, param_dict)

In [ ]:
from qiskit import QuantumCircuit
def generate_qaoa_circuit(theta):
    assert len(theta) % 2 == 0, "Error in parameters (Beta, Gamma)"

    nqubits = len(edges)  # Tantos qubits como aristas tenga el grafo
    circuit = QuantumCircuit(nqubits)

    layers = int(len(theta) / 2)
    beta = theta[:layers]
    gamma = theta[layers:]

    # |v0>
    for i in range(nqubits):
        circuit.h(i)

    for p in range(layers):
        circuit.barrier()

        # Hp
        for q_idx, coef in enumerate(linear_coefs):
            circuit.rz(coef, q_idx)
        for q_idxs, coef in quadra_coefs.items():
            circuit.rzz(coef * gamma[p], q_idxs[0], q_idxs[1])

        circuit.barrier()

        # Hm
        for q_idx in range(nqubits):
            circuit.rx(beta[p] * 2, q_idx)

    circuit.measure_all()
    return circuit

In [ ]:
def compute_expectation(counts):
    media = 0
    len_count = 0
    for bits, count in counts.items():
        cost = eval_cost_function(bits)
        media += cost * count
        len_count += count

    return media/len_count

In [ ]:
# Simulador
from qiskit import Aer
from scipy.optimize import minimize

backend = Aer.get_backend('aer_simulator')
shots = 1024

def execute_circuit(theta):
    qc = generate_qaoa_circuit(theta)
    counts = backend.run(qc, shots=shots).result().get_counts()
    return compute_expectation(counts)

num_layers = 10
theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
theta_res

In [ ]:
from qiskit.visualization import plot_histogram

backend = Aer.get_backend('aer_simulator')
shots = 1024

qc = generate_qaoa_circuit(theta_res.x)
counts = backend.run(qc, shots=shots).result().get_counts()
plot_histogram(counts, figsize=(7, 7))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def graph():
    x = np.arange(0.3, 6, 0.01)
    y = []
    beta = 1.0
    for gamma in x:
        y.append(execute_circuit([beta, gamma]))

    plt.plot(x, y)
    plt.show()

graph()

In [ ]:
# Statistics
statistics = {}
for generation in range(0, 1000):
    num_layers = 3
    theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
    qc = generate_qaoa_circuit(theta_res.x)
    counts = backend.run(qc, shots=shots).result().get_counts()

    path = max(counts, key=counts.get)
    if path not in statistics:
        statistics[path] = 0
    statistics[path] += 1

print(statistics)

In [ ]:
!dunstify -t $((30*1000)) --urgency=critical "Ejecución completada"